### **Import the necessary libraries**

In [2]:
import dash
from dash import dcc, html, dash_table
import plotly.express as px
import pandas as pd
import folium
import numpy as np
import requests
import math
from dash.dependencies import Input, Output

In [3]:
# Load Camp Coordinates & Area (AOR.xlsx)
aor_data = pd.read_excel("AOR.xlsx")

# Load Fire Susceptibility Data (Fire Susceptibility Data Block.csv)
fire_data = pd.read_csv("Fire Susceptability Data Block.csv")


In [4]:
# Standardize column names for merging
aor_data.rename(columns={'New_Camp_Name': 'CampName'}, inplace=True)

In [5]:
# Merge datasets on 'CampName'
merged_data = pd.merge(fire_data, aor_data, on='CampName', how='left')

In [6]:
# Compute Fire Susceptibility Index (FSI)
merged_data['FSI_Calculated'] = (
    merged_data['Environment'] + merged_data['Fuel'] +
    merged_data['Behaviour'] + merged_data['Response']
) / 4

In [8]:
print(merged_data)

   CampName Camp_Code CampSSID BlockName    Block Blocklet    Block_SSID  \
0   Camp 8E      C08E  CXB-210    C08E_A  Block A        A  CXB-210_A054   
1   Camp 8E      C08E  CXB-210    C08E_B  Block B        B  CXB-210_B055   
2   Camp 8E      C08E  CXB-210    C08E_C  Block C        C  CXB-210_C056   
3   Camp 8E      C08E  CXB-210    C08E_D  Block D        D  CXB-210_D057   
4   Camp 8E      C08E  CXB-210    C08E_E  Block E        E  CXB-210_E058   
..      ...       ...      ...       ...      ...      ...           ...   
82  Camp 24       C24  CXB-233     C24_E  Block E        E  CXB-233_E143   
83  Camp 24       C24  CXB-233     C24_F  Block F        F  CXB-233_F144   
84  Camp 24       C24  CXB-233     C24_G  Block G        G  CXB-233_G145   
85  Camp 25       C25  CXB-017     C25_A  Block A        A  CXB-017_A146   
86  Camp 25       C25  CXB-017     C25_B  Block B        B  CXB-017_B147   

    Environment       Fuel  Behaviour  ...  Upazila         Union   Latitude  \
0     3

In [ ]:
# Remove rows with missing Latitude/Longitude
cleaned_data = merged_data.dropna(subset=['Latitude', 'Longitude'])

# Fire Risk Classification
def classify_fsi(fsi):
    if fsi >= 67:
        return "Urgent"
    elif fsi >= 33:
        return "High"
    else:
        return "Moderate"

cleaned_data["FSI_Class"] = cleaned_data["FSI_Calculated"].apply(classify_fsi)

Step 2

In [1]:
def get_fwi_standard(lat, lon):
    url = f"https://wttr.in/{lat},{lon}?format=j1"
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        weather = response.json()
        temp = float(weather['current_condition'][0]['temp_C'])
        rh = float(weather['current_condition'][0]['humidity'])
        wind = float(weather['current_condition'][0]['windspeedKmph'])
        rain = float(weather['current_condition'][0].get('precipMM', 0))
        return round(calc_fwi_standard(temp, rh, wind, rain), 1)
    except Exception as e:
        print(f"Error fetching standard FWI: {e}")
        return 0

In [9]:
print(get_fwi_standard)

<function get_fwi_standard at 0x0000021A4778AC10>


In [ ]:
# ---------------- STEP 2: FETCH REAL-TIME FIRE WEATHER INDEX (FWI) ----------------
API_KEY = "YOUR_OPENWEATHER_API_KEY"

def get_fwi(lat, lon):
    """Fetch real-time weather & compute Fire Weather Index (FWI)."""
    url = f"https://api.openweathermap.org/data/2.5/weather?lat={lat}&lon={lon}&appid={API_KEY}&units=metric"
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        weather = response.json()
        
        temp = weather['main']['temp']
        humidity = weather['main']['humidity']
        wind = weather['wind']['speed']
        precipitation = weather.get('rain', {}).get('1h', 0)

        wind = max(wind, 0.1)
        precipitation = max(precipitation, 0.1)
        FFMC = 59.5 * math.exp(-0.1386 * humidity) + 5.3 * math.sqrt(wind) - 0.0365 * precipitation
        DMC = 1.5 * (temp - humidity) + 10 * math.log(precipitation)
        FWI = round(0.1 * FFMC + 0.5 * DMC, 2)

        return max(FWI, 0)
    except:
        return None

cleaned_data["FWI"] = cleaned_data.apply(lambda row: get_fwi(row["Latitude"], row["Longitude"]), axis=1)

In [ ]:
# ---------------- STEP 3: DASHBOARD LAYOUT ----------------
app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1("Fire Risk Analysis Dashboard", style={"text-align": "center"}),

    # Site Details & Fire Susceptibility Index
    html.Div([
        html.Div([
            html.H4("Site Details"),
            html.P("Site Name: Camp 8W, Cox’s Bazar"),
            html.P("Assessment Date: 2024-09-25"),
            html.P("Site Population: 33,515"),
        ], className="four columns"),

        html.Div([
            html.H4("Site Susceptibility Index"),
            html.H2("52 - High", style={"color": "orange"}),
        ], className="four columns", style={"background-color": "#F4A261", "padding": "20px"}),

        html.Div([
            html.H4("Fire Weather Index"),
            html.H2("0 ( - )", style={"color": "green"}),
        ], className="four columns", style={"background-color": "#A7C957", "padding": "20px"}),

    ], className="row"),

    # Fire Susceptibility Dimensions
    html.Div([
        html.H4("Susceptibility Dimensions"),
        dcc.Graph(
            figure=px.bar(
                x=["Response", "Behavior", "Fuel", "Environment"],
                y=[47, 51, 41, 68],
                labels={"x": "Dimensions", "y": "Score"},
                title="Fire Risk Factors"
            )
        ),
    ], className="six columns"),

    # Fire Risk Trends - FWI Time Series (Placeholder)
    html.Div([
        html.H4("Yearly Mean FWI"),
        dcc.Graph(
            figure=px.line(
                x=pd.date_range(start="2024-01-01", periods=12, freq="M"),
                y=np.random.randint(10, 35, size=12),
                labels={"x": "Month", "y": "FWI"},
                title="Yearly Mean FWI"
            )
        ),
    ], className="six columns"),

    # Fire Risk Map
    html.Div([
        html.H4("Fire Risk Map"),
        dcc.Graph(
            figure=px.scatter_mapbox(
                cleaned_data,
                lat="Latitude", lon="Longitude",
                color="FSI_Calculated",
                size="FSI_Calculated",
                hover_name="CampName",
                mapbox_style="carto-positron",
                zoom=10,
                title="Camp Fire Susceptibility"
            )
        ),
    ], className="twelve columns"),

    # Fire Susceptibility Indicator Scores
    html.Div([
        html.H4("Fire Susceptibility Indicator Scores"),
        dash_table.DataTable(
            columns=[{"name": col, "id": col} for col in ["CampName", "FSI_Calculated", "FSI_Class"]],
            data=cleaned_data.to_dict('records'),
            style_table={'overflowX': 'auto'}
        ),
    ], className="six columns"),

], className="container")

# ---------------- STEP 4: RUN DASH APP ----------------
if __name__ == '__main__':
    app.run_server(debug=True)